# Readme

This runs after the IMDB-Kaggle joined dataset (part-00000-b7bb5828-f78a-4d39-a019-33f1586faf35-c000.csv)
This pivots valid IMDB principals

# Import data

Check status at http://localhost:4040/

In [1]:
import os
memory = '2g'
cores = '4'
pyspark_submit_args = ' --driver-memory ' + memory + ' --executor-memory ' + memory + ' --executor-cores ' + cores + ' pyspark-shell'
# pyspark_submit_args = ' --driver-memory ' + memory + ' --executor-memory ' + memory + ' --executor-cores ' + cores + ' --bindAddress  127.0.0.1 pyspark-shell'
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [2]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql import DataFrame
import os
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.functions import lit
from pyspark.sql.functions import col, collect_list, concat_ws

In [3]:
spark = SparkSession.builder \
    .appName("project") \
    .config("spark.driver.bindAddress", "127.0.0.1")\
    .config("spark.sql.pivotMaxValues", "99999")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/13 22:29:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
#files = dbutils.fs.ls("dbfs:/FileStore/tables/")
files = os.listdir("./data")
for file in files:
    print(file)

.DS_Store
kaggle
spark-export
imdb-scrape
imdb
export


In [5]:
def add_prefix_to_columns(df: DataFrame, prefix: str) -> DataFrame:
    return df.select([col(c).alias(prefix + "_" + c) for c in df.columns])

## Import Kaggle

## Import IMDB Non-Commerical

In [6]:
imdb_principals = spark.read.option("header", "true")\
    .option("delimiter", "\t")\
    .option("inferSchema", "true")\
    .csv("data/imdb/title.principals.tsv.gz")

prefix = "imdb_principals"
imdb_principals = add_prefix_to_columns(imdb_principals, prefix)

imdb_name_basics = spark.read.option("header", "true")\
    .option("delimiter", "\t")\
    .option("inferSchema", "true")\
    .csv("data/imdb/name.basics.tsv.gz")

prefix = "imdb_name_basics"
imdb_name_basics = add_prefix_to_columns(imdb_name_basics, prefix)



24/03/13 22:29:52 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Import IMDB Scrape from Justin Huang

In [7]:
imdb_scrape = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .csv("data/imdb-scrape/imdb_scraped_datav2.csv.gz")
    #.csv("dbfs:/FileStore/tables/imdb_scraped_datav2_csv.gz")

prefix = "imdb_scrape"
imdb_scrape = add_prefix_to_columns(imdb_scrape, prefix)

In [8]:
part1_data = spark.read.option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", "|") \
    .csv("data/export/imdb_imdbscrape_kaggle_master.csv")

# Inspect data

## Part 1 Data

In [9]:
# part1_data.printSchema()
part1_data.show(n=5)
x = part1_data.count()
print(x)

24/03/13 22:30:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------+---------------------+--------------------------+-------------------+-------------------+-----------------+----------------------+------------------+----------------+--------------------+--------------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------------------------+---------------+--------------------+--------------------+-------------+--------------------+--------------------+--------------+
|imdb_scrape_imdb_id|imdb_title_attributes|imdb_title_isOriginalTitle|imdb_title_language|imdb_title_ordering|imdb_title_region|      imdb_title_title|imdb_title_titleId|imdb_title_types|         anime_Aired|anime_Duration|  anime_English name|      anime_Episodes|anime_Favorites|        anime_Genres|     anime_Image UR

## IMDB Data

In [10]:
imdb_name_basics.printSchema()
imdb_name_basics.show(n=5)

root
 |-- imdb_name_basics_nconst: string (nullable = true)
 |-- imdb_name_basics_primaryName: string (nullable = true)
 |-- imdb_name_basics_birthYear: string (nullable = true)
 |-- imdb_name_basics_deathYear: string (nullable = true)
 |-- imdb_name_basics_primaryProfession: string (nullable = true)
 |-- imdb_name_basics_knownForTitles: string (nullable = true)



+-----------------------+----------------------------+--------------------------+--------------------------+----------------------------------+-------------------------------+
|imdb_name_basics_nconst|imdb_name_basics_primaryName|imdb_name_basics_birthYear|imdb_name_basics_deathYear|imdb_name_basics_primaryProfession|imdb_name_basics_knownForTitles|
+-----------------------+----------------------------+--------------------------+--------------------------+----------------------------------+-------------------------------+
|              nm0000001|                Fred Astaire|                      1899|                      1987|              soundtrack,actor,...|           tt0027125,tt00723...|
|              nm0000002|               Lauren Bacall|                      1924|                      2014|                actress,soundtrack|           tt0075213,tt00373...|
|              nm0000003|             Brigitte Bardot|                      1934|                        \N|            

In [11]:
imdb_principals.printSchema()
imdb_principals.show(n=5)

root
 |-- imdb_principals_tconst: string (nullable = true)
 |-- imdb_principals_ordering: integer (nullable = true)
 |-- imdb_principals_nconst: string (nullable = true)
 |-- imdb_principals_category: string (nullable = true)
 |-- imdb_principals_job: string (nullable = true)
 |-- imdb_principals_characters: string (nullable = true)

+----------------------+------------------------+----------------------+------------------------+--------------------+--------------------------+
|imdb_principals_tconst|imdb_principals_ordering|imdb_principals_nconst|imdb_principals_category| imdb_principals_job|imdb_principals_characters|
+----------------------+------------------------+----------------------+------------------------+--------------------+--------------------------+
|             tt0000001|                       1|             nm1588970|                    self|                  \N|                  ["Self"]|
|             tt0000001|                       2|             nm0005690|        

## IMDB Scrape

In [12]:
imdb_scrape.printSchema()
imdb_scrape.show(n=5)

root
 |-- imdb_scrape__c0: string (nullable = true)
 |-- imdb_scrape_title: string (nullable = true)
 |-- imdb_scrape_description: string (nullable = true)
 |-- imdb_scrape_url: string (nullable = true)
 |-- imdb_scrape_image: string (nullable = true)
 |-- imdb_scrape_rating_value: string (nullable = true)
 |-- imdb_scrape_genre: string (nullable = true)
 |-- imdb_scrape_content_rating: string (nullable = true)
 |-- imdb_scrape_creator: string (nullable = true)
 |-- imdb_scrape_main_cast: string (nullable = true)
 |-- imdb_scrape_keywords: string (nullable = true)
 |-- imdb_scrape_duration: string (nullable = true)
 |-- imdb_scrape_soundtrack_trackname: string (nullable = true)
 |-- imdb_scrape_lyrics: string (nullable = true)
 |-- imdb_scrape_music: string (nullable = true)
 |-- imdb_scrape_arrangement: string (nullable = true)
 |-- imdb_scrape_performed_by: string (nullable = true)
 |-- imdb_scrape_imdb_id: string (nullable = true)

+---------------+--------------------+-------------

24/03/13 22:30:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , title, description, url, image, rating_value, genre, content_rating, creator, main_cast, keywords, duration, soundtrack_trackname, lyrics, music, arrangement, performed_by, imdb_id
 Schema: _c0, title, description, url, image, rating_value, genre, content_rating, creator, main_cast, keywords, duration, soundtrack_trackname, lyrics, music, arrangement, performed_by, imdb_id
Expected: _c0 but found: 
CSV file: file:///Users/jonathanlin/Documents/cse6242/project/data/imdb-scrape/imdb_scraped_datav2.csv.gz


# Clean data

## IMDB Scrape data

In [13]:
imdb_scrape.printSchema()
imdb_scrape_row_count = imdb_scrape.count()
print(imdb_scrape_row_count)

root
 |-- imdb_scrape__c0: string (nullable = true)
 |-- imdb_scrape_title: string (nullable = true)
 |-- imdb_scrape_description: string (nullable = true)
 |-- imdb_scrape_url: string (nullable = true)
 |-- imdb_scrape_image: string (nullable = true)
 |-- imdb_scrape_rating_value: string (nullable = true)
 |-- imdb_scrape_genre: string (nullable = true)
 |-- imdb_scrape_content_rating: string (nullable = true)
 |-- imdb_scrape_creator: string (nullable = true)
 |-- imdb_scrape_main_cast: string (nullable = true)
 |-- imdb_scrape_keywords: string (nullable = true)
 |-- imdb_scrape_duration: string (nullable = true)
 |-- imdb_scrape_soundtrack_trackname: string (nullable = true)
 |-- imdb_scrape_lyrics: string (nullable = true)
 |-- imdb_scrape_music: string (nullable = true)
 |-- imdb_scrape_arrangement: string (nullable = true)
 |-- imdb_scrape_performed_by: string (nullable = true)
 |-- imdb_scrape_imdb_id: string (nullable = true)

53786


In [14]:
imdb_scrape_unique = imdb_scrape.select("imdb_scrape_imdb_id", "imdb_scrape_title").distinct()
imdb_scrape_unique_row_count = imdb_scrape_unique.count()
print(imdb_scrape_unique_row_count)


18957


In [15]:
imdb_scrape_music_unique_notnull = imdb_scrape_unique.filter(
    col("imdb_scrape_soundtrack_trackname").isNotNull() |
    col("imdb_scrape_lyrics").isNotNull() |
    col("imdb_scrape_music").isNotNull() |
    col("imdb_scrape_arrangement").isNotNull() |
    col("imdb_scrape_performed_by").isNotNull())

imdb_scrape_music_unique_notnull.show(n = 10)

imdb_scrape_music_unique_notnull_row_count = imdb_scrape_music_unique_notnull.count()
print(imdb_scrape_music_unique_notnull_row_count)

+--------------------+--------------------+
| imdb_scrape_imdb_id|   imdb_scrape_title|
+--------------------+--------------------+
|                NULL| Megane and his g...|
|                NULL|                ..."|
|                NULL|https://www.imdb....|
|                NULL|https://www.imdb....|
|                NULL|https://www.imdb....|
|                NULL|https://www.imdb....|
|                NULL|https://www.imdb....|
|Arrangements by H...|            Wonabîzu|
|GODDESS FAMILY CL...|     Aa! Megamisama!|
|           tt0053275|Shônen Sarutobi S...|
+--------------------+--------------------+
only showing top 10 rows

10160


## Subset imdb name_basics and principals

In [16]:
# Perform leftsemi join on imdb_principals and part1_data

imdb_principals_part1_data = imdb_principals.join(part1_data, col("imdb_principals_tconst") == col("imdb_title_titleId"), "leftsemi")

imdb_principals_part1_data.show(n=5)

+----------------------+------------------------+----------------------+------------------------+-------------------+--------------------------+
|imdb_principals_tconst|imdb_principals_ordering|imdb_principals_nconst|imdb_principals_category|imdb_principals_job|imdb_principals_characters|
+----------------------+------------------------+----------------------+------------------------+-------------------+--------------------------+
|             tt0051694|                      10|             nm1281137|                producer|           producer|                        \N|
|             tt0051694|                       1|             nm0605662|                   actor|                 \N|      ["Xu Xian","Panda...|
|             tt0051694|                       2|             nm0594344|                 actress|                 \N|      ["Bai Niang","Xia...|
|             tt0051694|                       3|             nm3761746|                 actress|                 \N|             

In [17]:
imdb_principals_part1_data_distinct = imdb_principals_part1_data.select("imdb_principals_tconst").distinct()
x = imdb_principals_part1_data_distinct.count()
print(x)

4501


In [18]:
# Join imdb_principals_part1_data with imdb_name_basics

imdb_name_basics_scoped = imdb_name_basics.join(imdb_principals_part1_data, col("imdb_name_basics_nconst") == col("imdb_principals_nconst"), "leftsemi")
imdb_name_basics_scoped.show(n=5)

+-----------------------+----------------------------+--------------------------+--------------------------+----------------------------------+-------------------------------+
|imdb_name_basics_nconst|imdb_name_basics_primaryName|imdb_name_basics_birthYear|imdb_name_basics_deathYear|imdb_name_basics_primaryProfession|imdb_name_basics_knownForTitles|
+-----------------------+----------------------------+--------------------------+--------------------------+----------------------------------+-------------------------------+
|              nm0000354|                  Matt Damon|                      1970|                        \N|              producer,actor,so...|           tt3659388,tt01192...|
|              nm0015085|                Kazuki Akane|                      1962|                        \N|              director,writer,a...|           tt11008522,tt3354...|
|              nm0259074|               Juli Erickson|                      1939|                        \N|            

In [19]:
# Join the subset data together (inner join imdb_principals_part1_data and name_basics) and then pivot on imdb_principals and pivot the data on imdb_principals_tconst, rolling up all the imdb_name_basics_primaryName

imdb_principals_name_basics = imdb_principals_part1_data.join(imdb_name_basics_scoped, col("imdb_principals_nconst") == col("imdb_name_basics_nconst"), "inner").select("imdb_principals_tconst", "imdb_name_basics_primaryName")
imdb_principals_name_basics.show(n=5)

+----------------------+----------------------------+
|imdb_principals_tconst|imdb_name_basics_primaryName|
+----------------------+----------------------------+
|             tt0876563|                  Matt Damon|
|             tt3354096|                Kazuki Akane|
|             tt0872308|                Kazuki Akane|
|             tt0270933|                Kazuki Akane|
|             tt3496018|               Juli Erickson|
+----------------------+----------------------------+
only showing top 5 rows



In [20]:
x = imdb_principals_name_basics.count()
print(x)

40357


In [21]:
# (imdb_principals_name_basics\
#   .coalesce(1)
#   .write\
#   .mode('overwrite')\
#   .option("header", "true")\
#   .option("quoteAll", "true")\
#   .option("delimiter", "|")\
#   .option("nullValue", "")\
#   .csv("data/export/imdb_principals_name_basics.csv"))

Sample check

In [22]:
# filter imdb_principals_name_basics by imdb_principals_tconst == tt0051694 or tt0058339

imdb_principals_name_basics_sample = imdb_principals_name_basics.filter((col("imdb_principals_tconst") == "tt0051694") | (col("imdb_principals_tconst") == "tt0058339"))
imdb_principals_name_basics_sample.show()

+----------------------+----------------------------+
|imdb_principals_tconst|imdb_name_basics_primaryName|
+----------------------+----------------------------+
|             tt0058339|                Adolf Hitler|
|             tt0051694|             Alvin Schoncite|
|             tt0058339|          Shinsuke Chikaishi|
|             tt0058339|                Osamu Tezuka|
|             tt0051694|                 Shin Uehara|
|             tt0051694|             Seiichi Yashiro|
|             tt0051694|                     Lisa Lu|
|             tt0051694|              Hiroshi Ohkawa|
|             tt0051694|           Virginia Blackman|
|             tt0051694|            Hisaya Morishige|
|             tt0051694|             Taiji Yabushita|
|             tt0051694|               Mariko Miyagi|
|             tt0051694|                   Ben Pivar|
+----------------------+----------------------------+



In [23]:
# pivot the data on imdb_principals_tconst, rolling up all the imdb_name_basics_primaryName. 

# x = imdb_principals_name_basics_sample.groupBy("imdb_principals_tconst").pivot("imdb_principals_tconst").agg(concat_ws(", ", collect_list("imdb_name_basics_primaryName")))
x = imdb_principals_name_basics_sample.groupBy("imdb_principals_tconst").agg(concat_ws(", ", collect_list("imdb_name_basics_primaryName")).alias("imdb_name_basics_primaryName"))
x.show(n = 5, truncate = False)

+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|imdb_principals_tconst|imdb_name_basics_primaryName                                                                                                                          |
+----------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|tt0058339             |Adolf Hitler, Shinsuke Chikaishi, Osamu Tezuka                                                                                                        |
|tt0051694             |Alvin Schoncite, Shin Uehara, Seiichi Yashiro, Lisa Lu, Hiroshi Ohkawa, Virginia Blackman, Hisaya Morishige, Taiji Yabushita, Mariko Miyagi, Ben Pivar|
+----------------------+------------------------------------------------------------------------------------------------

Process

In [24]:

imdb_principals_name_basics_pivot = imdb_principals_name_basics.groupBy("imdb_principals_tconst").agg(concat_ws(", ", collect_list("imdb_name_basics_primaryName")).alias("imdb_name_basics_primaryName"))

In [25]:
imdb_principals_name_basics_pivot.show()

+----------------------+----------------------------+
|imdb_principals_tconst|imdb_name_basics_primaryName|
+----------------------+----------------------------+
|             tt0051694|        Alvin Schoncite, ...|
|             tt0053275|        Teruo Miyazaki, M...|
|             tt0054265|        Daisaku Shirakawa...|
|             tt0054631|        Jô Mizuki, Isuzu ...|
|             tt0057669|        Yukiko Okada, Mit...|
|             tt0058339|        Adolf Hitler, Shi...|
|             tt0058342|                Osamu Tezuka|
|             tt0058734|        Daisaku Shirakawa...|
|             tt0058807|        Sonia Owens, Pete...|
|             tt0059212|        Chiyoko Honma, Hi...|
|             tt0060926|        Judy Ongg, Arihir...|
|             tt0061069|        Takashi Mitsui, M...|
|             tt0061077|        Noriyoshi Matsûra...|
|             tt0061300|        Jack Curtis, Cleo...|
|             tt0062044|        Hôsei Komatsu, Ma...|
|             tt0062260|    

In [26]:
x = imdb_principals_name_basics_pivot.count()
print(x)

4501


In [27]:
(imdb_principals_name_basics_pivot\
  .coalesce(1)
  .write\
  .mode('overwrite')\
  .option("header", "true")\
  .option("quoteAll", "true")\
  .option("delimiter", "|")\
  .option("nullValue", "null")\
  .csv("data/export/imdb_principals_name_basics_pivot.csv"))

In [28]:
# AnalysisException: The pivot column imdb_principals_tconst has more than 10000 distinct values, this could indicate an error. If this was intended, set spark.sql.pivotMaxValues to at least the number of distinct values of the pivot column.
spark.stop()
